In [18]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

import os

# Bulding Smart Voice Assistant

In [9]:
import speech_recognition as sr
import elevenlabs

elevenlabs.set_api_key(os.getenv('ELEVEN_API_KEY'))

class AudioInterface:
    def listen(self) -> str:
        recognizer = sr.Recognizer()

        with sr.Microphone() as source:
            print("Say something...")
            audio = recognizer.listen(source)

        text = recognizer.recognize_whisper_api(
            audio,
            api_key=os.getenv('OPENAI_API_KEY'),
        )

        return text
    
    def speak(self, text: str) -> None:
        audio = elevenlabs.generate(
            text=text,
            model="eleven_monolingual_v1"
        )

        elevenlabs.play(audio)
    


In [11]:
interface = AudioInterface()
text = interface.listen()
print(text)

interface.speak(text)

Say something...
My name is Yuri.


# Building conversation agent

In [13]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

class ConversationAgent:
    def __init__(self):
        self.llm = ChatOpenAI()
        self.chain = ConversationChain(llm=self.llm)
    
    def run(self, text):
        return self.chain.run(text)

In [16]:
agent = ConversationAgent()

interface = AudioInterface()

while True:
    text = interface.listen()
    print("ME: " + text)
    response = agent.run(text)
    print("AI: " + response)
    interface.speak(response)

    


Say something...
ME: Hey, how's it going?
AI: Hello! I'm doing well, thank you for asking. I'm an AI designed to assist with various tasks and engage in conversation. How can I help you today?
Say something...
ME: What's the weather today in Chicago?
AI: I'm sorry, but I don't have access to real-time information. However, you can easily check the weather in Chicago by searching for it online or using a weather app on your phone.
Say something...
ME: What's the typical winter like in Chicago?
AI: Chicago experiences cold and snowy winters. The average temperature in January, the coldest month, is around 30°F (-1°C). However, temperatures can drop below freezing, and wind chills can make it feel even colder. Chicago receives an average of 36 inches (91 cm) of snowfall during the winter season. It's important to bundle up and be prepared for winter weather conditions if you're planning to visit or live in Chicago during the winter months.


KeyboardInterrupt: 

# Augmenting the Agent with Tools

In [22]:
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.callbacks import StdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
class SmartChatAgent:
    def __init__(self):
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
        )

        self.llm = ChatOpenAI()
        self.tools = load_tools(["google-search"])

        self.agent = initialize_agent(
            llm=self.llm,
            tools=self.tools,
            agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def run(self, text):
        return self.agent.run(text, callbacks=[StdOutCallbackHandler()])

In [24]:
agent = SmartChatAgent()

interface = AudioInterface()

while True:
    text = interface.listen()
    print("ME: " + text)
    response = agent.run(text)
    print("AI: " + response)
    interface.speak(response)

    

Say something...
ME: Hi, how are you? My name is Yuri.


> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

google_search: A wrapper around Google Search. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [google_search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Hi, how are you? My name is Yuri.
Thought:

> Finished chain.
This question does not require any external tools to answer.
Observation: Invalid Format: Missing 'A

KeyboardInterrupt: 